# Quadrotor Modeling
citing https://nbviewer.jupyter.org/github/plusk01/nonlinearquad/blob/master/quadrotor_model.ipynb

## Coordinate Frames
Coordinate frames are rigid, orthonormal bases of $mathbb{R}^3$.  
Tow main frames:  
* Inertial frame $\mathcal{F}^i$  
* body frame $\mathcal{F}^b$  

**Intermediate frame** exist to help us account for the roll, pitch and yaw rotation of the drone.
* The first intermediate frame is the vehicle frame $\mathcal{F}^v$, unrolled, unpitched, unyawed.
* Additional, camera frame, gimbal frame, or other sensor ormeansurement frames.  

The vehicle frame leads to the vehicle-1 frame $\mathcal{F}^{v1}$ by yawing about the $k^v$ axis, resulting in the unrolled, unpitched frame. Pitching about the new $j^{v1}$ to vehicle-2 frame $\mathcal{F}^{v2}$, and rolling about the new $i^{v2}$ results in the properly oriented body frame $\mathcal{F}^b$.   
Passive, right-handed rotations. 3-2-1($\psi-\theta-\phi$) Euler angles for yaw, pich and roll, respectively.  

Rotation matrices in $\mathbb{R}^{3\times 3}$ are built below.


In [17]:
import numpy as np

In [19]:
def Rot_v_to_v1(psi):
    R = np.array([
        [ np.cos(psi), np.sin(psi), 0],
        [-np.sin(psi), np.cos(psi), 0],
        [     0,           0    ,   1]
    ])
    return R

def Rot_v1_to_v2(theta):
    R = np.array([
        [np.cos(theta), 0, -np.sin(theta)],
        [      0      , 1,        0      ],
        [np.sin(theta), 0,  np.cos(theta)]
    ])
    return R

def Rot_v2_to_b(phi):
    R = np.array([
        [1,       0,           0     ],
        [0,  np.cos(phi), np.sin(phi)],
        [0, -np.sin(phi), np.cos(phi)]
    ])
    return R

In [20]:
def Rot_v_to_b(phi, theta, psi):
    return Rot_v2_to_b(phi).dot(Rot_v1_to_v2(theta).dot(Rot_v_to_v1(psi)))

def Rot_i_to_b(phi, theta, psi):
    return Rot_v_to_b(phi, theta, psi)

## Configuration Space
The quadrotor equations of motion consisits of 6 degrees of freedom(DOF) and 12 ordinary differential equations(6 are kinematic and 6 are dynamic).  
The quadrotor is modelled  as a freely-rotating rigid body evolving in $\mathcal{M} = \mathbb{R}^3 \times \mathit{SO}(3) = \mathit{SE}(3)$. 
Where $\mathit{SO}(3)$ describes the orientation of rigid body in 3D space and is not isomorphic to $\mathbb{R}^3$. Parameterization: the intuitive Euler angles.
\begin{align}
\phi \in [ 0, 2\pi) \subset \mathbb{R}\\
\theta \in (-\pi/2,\pi/2)\subset\mathbb{R}\\
\psi \in [0, 2\pi)\subset \mathbb{R}
\end{align}  
Thus, the states of quadrotor isgiven by $\hbox{x} = [x\ \ y\ \ z\ \ \phi\ \ \theta\ \ \psi]^\top \in \mathbb{R}^6$.

## Equations of motion
### Kinematics
Kinematics describe the motion of an object without consideration of what caused the motion. Using kinematics we can relate position to velocity and orientation to angular velocity.
* These equations are irrespective of intertial properties such mass and size.


#### Translational
$$\dot r^i = R_b^i v^b$$

#### Rotational
Its about how the angular velocities relate to the current orientation.
A rate gyro is used to measure the angular velocity about each of the body-frame axes of the rigid-body.
These gyro measurements are given by:
\begin{align}
\omega = p \hbox{i} ^b+q \hbox{j} ^b+r \hbox{k} ^b = [p\ \ q\ \ r]^\top
\end{align}
**Each Euler angle is defined in different frame.** Therefore, thederivatives of each Euler angle must be transformed into the proper frame as follows:
\begin{align}
\omega = 
\left[
\begin{array}
\\p\\q\\r\\
\end{array}
\right]
&=
\left[
\begin{array}
\\\dot\phi\\0\\0\\
\end{array}
\right]+
R_{v2}^b(\phi)
\left[
\begin{array}
\\0\\\dot\theta\\0\\
\end{array}
\right]+
R_{v2}^b(\phi)R_{v1}^b(\theta)
\left[
\begin{array}
\\0\\0\\\dot\psi\\
\end{array}
\right]\\
&=
\left[
\begin{array}
\\\dot\phi\\0\\0\\
\end{array}
\right]+
\left[
\begin{matrix}
1 & 0 & 0\\
0 & cos\phi & sin\phi\\
0 & -sin\phi & cos\phi
\end{matrix}
\right]
\left[
\begin{array}
\\0\\\dot\theta\\0\\
\end{array}
\right]+
\left[
\begin{matrix}
1 & 0 & 0\\
0 & cos\phi & sin\phi\\
0 & -sin\phi & cos\phi
\end{matrix}
\right]
\left[
\begin{matrix}
cos\theta & 0 & -sin\theta\\
0 & 1 & 0\\
sin\theta & 0 & cos\theta
\end{matrix}
\right]
\left[
\begin{array}
\\0\\0\\\dot\psi\\
\end{array}
\right]\\
&=
\left[
\begin{matrix}
1 & 0 & -sin\theta\\
0 & cos\phi & sin\phi cos\theta\\
0 & -sin\phi & cos\phi cos\theta\\
\end{matrix}
\right]
\left[
\begin{array}
\\\dot\phi\\\dot\theta\\\dot\psi\\
\end{array}
\right]
\end{align}

Note: $\hbox{det}\Gamma^{-1}=cos\theta$$\Rightarrow$for $\theta\in(-\pi/2,\pi/2)$,the matrix $\Gamma^{-1}$ is invertible.  
We can write the rotational kinematic equation using Euler angles as  
\begin{align}
\dot \Phi=
\left[
\begin{array}
\\\dot\phi\\\dot\theta\\\dot\psi\\
\end{array}
\right]=
\left[
\begin{matrix}
1 & \sin\phi \tan\theta & \cos\phi \tan\theta\\
0 & \cos\phi & -\sin\phi\\
0 & \sin\phi \sec\theta & \cos\phi \sec\theta
\end{matrix}
\right]=
\Gamma(\phi,\theta)\omega
\end{align}

### Dynamics
Dynamics describe the motion of objects in relation to forces and torque, which cause motion.  
**This is how we use a controller to interact with physical system**.  
We can control acuators which cause a force or torque on the system.
#### Translational
Newton's second law only hold in inertial frame. Therefore, we use the equation of Cariolois to express the derivative of an inertial frame quality to the derivative in the body frame.
$$\hbox{F}_g-\hbox{T}=m\dot v^b + \omega\times v^b$$
Where $\hbox{F}_g=mg[-\sin\theta\ \ \cos\theta\sin\phi\ \ \cos\theta\cos\phi]^\top$, and $\hbox{T} = [0\ \ 0\ \ f]^\top$.  
Then
$$\dot v^b = \frac{1}{m} [\hbox{F}_g-\hbox{T}-\omega\times v^b]$$.  
#### translational(inertial)
We can express the trasnslational dynamic in the inertial frame, which can be advantagous if we are trying to command inertial position or velocities.  
\begin{align}
m \ddot {\hbox{r}} ^i 
= mg \hbox{k}^i - 
R_b^i
\left[
\begin{array}
\\0\\0\\u_1\\
\end{array}
\right]
\end{align}
#### tranlational(with drag)
More ralistic by adding drag. It is ideal to have a higher  fidelity plant model than the control/estimation models --- **This will be true  in the real life**.
\begin{align}
m \ddot {\hbox{r}} ^i 
= mg \hbox{k}^i - 
R_b^i
\left[
\begin{array}
\\0\\0\\u_1\\
\end{array}
\right] -
\left[
\begin{array}
\\\mu\\\mu\\0\\
\end{array}
\right]
\dot r^i
\end{align}
#### Rotational
Following the Newton's second law for rotation motion ($M=I\dot \omega$)
$$I\dot\omega=-\omega\times I\omega + M$$
We assume that the quadrotor is symmetric about its axes so that
\begin{align}
I =
\left[
\begin{matrix}
I_x & 0 & 0\\
0 & I_y & 0\\
0 & 0 & I_z\\
\end{matrix}
\right]
\end{align}

### Forces and Moments
We must determine the forces and moments that are used to drive the 6 DOF dynamic model we have derived.  
\begin{align}
\left[
\begin{array}
\\u_1\\u_2\\u_3\\u_4\\
\end{array}
\right]=
\left[
\begin{matrix}
1 & 1 & 1 & 1\\
0 & -l & 0 & l\\
l & 0 & -l & 0\\
-c_T & c_T & -c_T & c_T\\
\end{matrix}
\right]
\left[
\begin{array}
\\T_1\\T_2\\T_3\\T_4\\
\end{array}
\right]
\end{align}
The thrust $T_i$ of each motor is related to the propeller's angular velocity
$$T_i = k\Omega _i^2$$
Note: the thrust always overcome gravity, a positive $m_{\phi}$ yeilds a positive motion along $\hbox{j}^b$. A positive $m_{\theta}$ yeilds a megative motion along $\hbox{i}^b$.

In [21]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [22]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

In [25]:
class Quadrotor(object):
    # models a quadrotor evoving in SE(3)
    def __init__(self, r=None, Euller=None, r_dot=None, omega=None):
        # internal quadrotor states
        self.r = r if r is not None else np.zeros((3,1))
        self.Euller = Euller if Euller is not None else np.zeros((3,1))
        self.r_dot = r_dot if r_dot is not None else np.zeros((3,1))
        self.omega = omega if omega is not None else np.zeros((3,1))
        
        # physical parameters
        self.g = 9.81
        self.mass = 3.81
        Jxx = 0.06024
        Jyy = 0.122198
        Jzz = 0.132166
        self.I = np.array([[Jxx, 0, 0],
                         [0, Jyy, 0],
                         [0, 0, Jzz]])
        self.Mu = np.diag(np.array([0.85,0.85,0.85])) # drag
        # self.Mu = np.dia(np.array([0, 0, 0])) # drag
        
        self.Tmax = 40
        self.Mmax = 2 # max control actuations
        
        self.Niters= 0 # convenience
        
    def __str__(self):
        s = "Quadrotor state after {} ilter: \n".format(self.Nilters)
        s += "\tr:     {}.T\n".format(self.r.T)
        s += "\tPhi:   {}.T\n".foramt(self.Phi.T)
        s += "\tv      {}.T\n".format(self.v.T)
        s += "\tomega: {}.T\n".foramt(self.omega.T)
        return s
    
    @staticmethod
    def clamp(r_dot, limit):
        r_dot = np.copy(r_dot)
        r_dot[np.abs(r_dot) > limit] = np.sign(r_dot[np.abs(r_dot) > limit])*limit
        
    @staticmethod
    def Gamma(phi, theta):
        return np.array([
            [1, np.sin(phi)*np.tan(theta), np.cos(phi)*np.tan(theta)],
            [0, np.cos(phi), -np.sin(phi)],
            [0, np.sin(phi)*np.cos(theta), np.cos(phi)*np.cos(theta)]
        ])
    
    def Fg(self, phi, theta):
        return self.mass*self.g*np.array([
            [-np.sin(theta)],
            [np.cos(theta)*np.sin(phi)],
            [np.cos(theta)*np.cos(phi)]
        ])
    
    def update(self, u, dt):
        # iterations
        self.Niters += 1
        
        # extract Euler angles
        phi = self.Euller.flatten()[0]
        theta = self.Euller.flatten()[1]
        psi = self.Euller.flatten()[2]
        
        # Forces and Moments
        u = np.array(u).flatten()
        T = np.array([ [0, 0, u[0]] ]).T
        M = np.array([ [u[1], u[2], u[3]] ]).T
        
        # saturate control efforts
        T = self.clamp(T, self.Tmax)
        M = self.clamp(M, self.Mmax)
        
        # kinematics
        ## translational
        # f = lambda r: Rot_i_to_b(ph, th, ps).T.dot(self.v) # f=\dot r^i self.v=v^b
        f = lambda r: self.r_dot
        self.r = rk4(f, self.r, dt) # 4th rounge-kutta
        
        ## Rotational
        f = lambda Euller: self.Gamma(Euller[0], Euller[1]).dot(self.omega)
        self.Euller = rk4(f, self.Euller, dt)
        
        # Dynamics
        ## translational
        # f = lambda v: (1/self.mass)*(self.Fg(ph, th) - T - np.cross(self.omega, v, axis=0)) # body frame
        f = lambda r_dot: (1/self.mass)*(self.Fg(0, 0) - Rot_i_to_b(ph, th, ps).T.dot(T) - self.Mu.dot(r_dot)) # inertial frame
        self.r_dot = rk4(f, self.r_dot, dt)
        
        ## Rotational
        f = lambda omega: np,linalg.inv(self.I).dot((-np.cross(omega, self.I.dot(omega), axis=0) + M))
        self.omega = rk4(f, self.omega, dt)
        
        # update control input
        u = np.hstack((T[2], M.flatten()))
        return u